<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">DESPLIEGUE DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [1]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


#### CONEXIÓN CON LA BASE DE DATOS

In [ ]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db')

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [ ]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',), ('ratings_movies',)]

In [ ]:
pd.read_sql_query("SELECT * FROM movies", conexion)
display(pd.read_sql_query("SELECT * FROM ratings", conexion))

In [3]:
import pandas as pd
import sqlite3 as sql
import logging
from pathlib import Path
from google.colab import drive

# 1. Montar Google Drive (sólo si estás en Google Colab)
drive.mount('/content/drive')

# 2. Configuración de rutas usando pathlib (ajusta según tu estructura)
base_path = Path("/content/drive/MyDrive/marketing/Marketing")
db_file = base_path / "data/db_movies_copy.db"  # Asegúrate de que la extensión sea .db si aplica
output_csv = base_path / "salidas/recomendaciones/peliculas_mas_populares.csv"
log_file = base_path / "salidas/recomendaciones/popular_log.log"

# 3. Configurar logging
logging.basicConfig(
    filename=str(log_file),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def obtener_top_peliculas_populares(db_path, top_n=10):
    """
    Obtiene las películas con mayor número de calificaciones.
    """
    logging.info("Inicio del proceso para obtener películas más populares")
    conn = None
    try:
        conn = sql.connect(str(db_path))
        query = f"""
        SELECT m.titulo, COUNT(r.movieId) AS num_ratings
        FROM movies m
        JOIN ratings r ON m.movieId = r.movieId
        GROUP BY m.titulo
        ORDER BY num_ratings DESC
        LIMIT {top_n};
        """
        df = pd.read_sql_query(query, conn)
        logging.info(f"Se recuperaron {len(df)} películas populares exitosamente")
        return df
    except Exception as e:
        logging.error(f"Error al obtener películas populares: {e}")
        print(f"Error al obtener películas populares: {e}")
        return pd.DataFrame()
    finally:
        if conn:
            conn.close()

def guardar_populares_en_csv(df, filepath):
    """
    Guarda el DataFrame en un archivo CSV.
    """
    try:
        df.to_csv(filepath, index=False, encoding='utf-8')
        logging.info(f"Películas populares guardadas en: {filepath}")
        print(f"Películas populares guardadas en: {filepath}")
    except Exception as e:
        logging.error(f"Error al guardar las películas populares en CSV: {e}")
        print(f"Error al guardar las películas populares en CSV: {e}")

# 4. Ejecutar
if __name__ == "__main__":
    top_peliculas_df = obtener_top_peliculas_populares(db_file, top_n=10)
    if not top_peliculas_df.empty:
        guardar_populares_en_csv(top_peliculas_df, output_csv)
    else:
        print("No se pudieron obtener las películas más populares.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Películas populares guardadas en: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares.csv


In [4]:
# ========================
# 1. IMPORTACIONES
# ========================
import pandas as pd
import sqlite3 as sql
import logging
from pathlib import Path
from google.colab import drive

# ========================
# 2. MONTAR GOOGLE DRIVE
# ========================
drive.mount('/content/drive')

# ========================
# 3. DEFINIR RUTAS
# ========================
base_path = Path("/content/drive/MyDrive/marketing/Marketing")
db_file = base_path / "data/db_movies_copy.db"
output_csv = base_path / "salidas/recomendaciones/peliculas_mas_populares.csv"
output_excel = base_path / "salidas/recomendaciones/peliculas_mas_populares.xlsx"
log_file = base_path / "salidas/recomendaciones/popular_log.log"

# Crear carpetas si no existen
output_csv.parent.mkdir(parents=True, exist_ok=True)
log_file.parent.mkdir(parents=True, exist_ok=True)

# ========================
# 4. CONFIGURAR LOGGING
# ========================
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Limpiar handlers previos en Colab
if logger.hasHandlers():
    logger.handlers.clear()

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(str(log_file))
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# ========================
# 5. FUNCIONES
# ========================

def obtener_top_peliculas_populares(db_path, top_n=10):
    """
    Obtiene las películas más populares por número de calificaciones.
    """
    logging.info("Inicio del proceso para obtener películas más populares")
    conn = None
    try:
        conn = sql.connect(str(db_path))
        query = f"""
        SELECT m.titulo, COUNT(r.movieId) AS num_ratings
        FROM movies m
        JOIN ratings r ON m.movieId = r.movieId
        GROUP BY m.title
        ORDER BY num_ratings DESC
        LIMIT {top_n};
        """
        df = pd.read_sql_query(query, conn)
        logging.info(f"Se recuperaron {len(df)} películas populares exitosamente")
        return df
    except Exception as e:
        logging.error(f"Error al obtener películas populares: {e}")
        print(f"Error al obtener películas populares: {e}")
        return pd.DataFrame()
    finally:
        if conn:
            conn.close()

def guardar_populares_en_csv(df, filepath):
    """
    Guarda el DataFrame en CSV.
    """
    try:
        df.to_csv(filepath, index=False, encoding='utf-8')
        logging.info(f"Películas populares guardadas en CSV: {filepath}")
        print(f"Películas populares guardadas en CSV: {filepath}")
    except Exception as e:
        logging.error(f"Error al guardar en CSV: {e}")
        print(f"Error al guardar en CSV: {e}")

def guardar_populares_en_excel(df, filepath):
    """
    Guarda el DataFrame en Excel.
    """
    try:
        df.to_excel(filepath, index=False, engine='openpyxl')
        logging.info(f"Películas populares guardadas en Excel: {filepath}")
        print(f"Películas populares guardadas en Excel: {filepath}")
    except Exception as e:
        logging.error(f"Error al guardar en Excel: {e}")
        print(f"Error al guardar en Excel: {e}")

# ========================
# 6. EJECUCIÓN PRINCIPAL
# ========================

top_peliculas_df = obtener_top_peliculas_populares(db_file, top_n=10)

if not top_peliculas_df.empty:
    guardar_populares_en_csv(top_peliculas_df, output_csv)
    guardar_populares_en_excel(top_peliculas_df, output_excel)
else:
    print("No se pudieron obtener las películas más populares.")


2025-05-12 03:36:41,489 - INFO - Inicio del proceso para obtener películas más populares
2025-05-12 03:36:41,498 - ERROR - Error al obtener películas populares: Execution failed on sql '
        SELECT m.title, COUNT(r.movie_id) AS num_ratings
        FROM movies m
        JOIN ratings r ON m.movie_id = r.movie_id
        GROUP BY m.title
        ORDER BY num_ratings DESC
        LIMIT 10;
        ': no such column: m.title


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error al obtener películas populares: Execution failed on sql '
        SELECT m.title, COUNT(r.movie_id) AS num_ratings
        FROM movies m
        JOIN ratings r ON m.movie_id = r.movie_id
        GROUP BY m.title
        ORDER BY num_ratings DESC
        LIMIT 10;
        ': no such column: m.title
No se pudieron obtener las películas más populares.
